In [ ]:
# Install every required package. Note that if you only want CUDA, you can skip PyOpenCL, clFFT and gpyfft
install_opencl = False
try:
  # Do not reinstall if pynx is already present (useful for 'run all')
  import pynx
except ModuleNotFoundError:
  # Base scientific computing packages and SILX
  !pip install numpy cython scipy matplotlib ipython scikit-image
  !pip install h5py hdf5plugin h5glance silx fabio
  # Install PyCUDA & PyOpenCL
  !pip install pycuda scikit-cuda
  # Install PyNX (nightly)
  !pip install http://ftp.esrf.fr/pub/scisoft/PyNX/pynx-devel-nightly.tar.bz2
  if install_opencl:
    !pip install opencl
    # Install clFFT
    !git clone https://github.com/clMathLibraries/clFFT.git --branch=v2.12.2
    !mkdir build
    !cd build && cmake ../clFFT/src && make all install
    # This is needed so that gpyfft can find libclFFT
    !ln -sf /usr/local/lib64/libclFFT.so.2 /usr/lib/
    # Install gpyfft (we need a specific version for PyNX compatibility)
    !git clone https://github.com/geggo/gpyfft.git
    !cd gpyfft && git checkout 2c07fa8e7674757 && python setup.py install
  
import numpy as np
import matplotlib.pyplot as plt
from pynx.scattering import Fhkl_thread
from pynx.utils.plot_utils import complex2rgbalog, complex2rgbalin

!pip install tifffile
!pip install tqdm
import tifffile as tif
from tqdm import tqdm
!pip install ovito

from ovito.io import import_file

import os
import numpy as np

In [15]:
def naming_convention(save_folder, inputs):
    """
    Parameters
    ----------
    save_folder : str
        The path to the folder where you want to save the data to.
    inputs : list
        standard list is (reflection, sampling_rate, file). This function can be
        redefined to fit the Users needs for folder structure. 
        
        reflection - the current reflection the iterative program is on
        sampling_rate - the current sampling rate the iterative program is on
        file - the current sample the iterative program is on
    """
    
    reflection, sampling_rate, file = inputs
    
    folder1 = f'[{reflection[0]}{reflection[1]}{reflection[2]}]'
    main_name = f'{file[:-4]}_reflection_{folder1}_sprate_{sampling_rate}'
    #main_folder = f'{save_folder}{folder1}'
    main_folder = f'{save_folder}'
    
    #fhkl_save_location = f'{main_folder}/{sampling_rate}/tif/{main_name}.tif'
    fhkl_save_location = f'{main_folder}{main_name}.tif'
    
    return main_name, main_folder, fhkl_save_location

def check_folder_exists(main_folder):
    """
    Parameters
    ----------
    main_folder : str
        the main save folder
    
    """
    
    if not os.path.exists(f'{main_folder}/'):
        os.mkdir(f'{main_folder}/')
        print('Created Main Folder')

def import_xyz_data(file):
    """Import data through ovito.io.import_file
    """
    
    pipeline = import_file(file)
    data = pipeline.source.compute(0)
    x = data.particles.positions[:, 0]
    y = data.particles.positions[:, 1]
    z = data.particles.positions[:, 2]

    x = x - np.mean(x)
    y = y - np.mean(y)
    z = z - np.mean(z)

    return x, y, z 
            
            
def get_simulation(input_folder,
                  file,
                  hkl=[1, 1, 1],
                  lattice=4.0782,
                  nstep=150,
                  nx=128,
                  ny=128,
                  nz=128):
    """Simulate through PyNX
    
    Parameters
    ----------
    input_folder : str
        folder location of file
        
    file : str
        name of file inside folder to load
        
    hkl : array
        the bragg reflection to calculate
        
    lattice : float
        lattice constant to use
        
    nstep : int
        number or steps for your data. What I call sampling rate
        
    nx, ny, nz : int
        the dimensions of the simulation
    
    """

    x, y, z, = import_xyz_data(input_folder+file)

    dq = 1/lattice/nstep


    dqx = (np.arange(nx)[:,np.newaxis,np.newaxis]-nx/2)*dq
    dqy = (np.arange(ny)[np.newaxis,:,np.newaxis]-ny/2)*dq
    dqz = (np.arange(nz)[np.newaxis,np.newaxis,:]-nz/2)*dq

    qx = hkl[0]/lattice+dqx
    qy = hkl[1]/lattice+dqy
    qz = hkl[2]/lattice+dqz

    fhkl, dt = Fhkl_thread(qx,qy,qz,
                        x,y,z,
                        occ=None,
                        gpu_name=gpu_name,
                        language=gpu_language)

    intensity = abs(fhkl)**2


    return fhkl, intensity

In [16]:
save_folder = '/content/drive/My Drive/Will_CDI_Project/edge/'
# List all the reflections you would like to simulate
reflections_list = [[-1,1,-1], ]

# List all the sampling rates your would like to simulate
sampling_list = [100, 125, 150, 200, 225]

naming_convention(save_folder, (reflections_list[0], sampling_list[0], 'testing.mp4'))

('testing_reflection_[-11-1]_sprate_100',
 '/content/drive/My Drive/Will_CDI_Project/edge/',
 '/content/drive/My Drive/Will_CDI_Project/edge/testing_reflection_[-11-1]_sprate_100.tif')

In [ ]:
# List all the reflections you would like to simulate
reflections_list = [[-1,1,-1], ]

# List all the sampling rates your would like to simulate
sampling_list = [100, 125, 150, 200, 225]

# Tell the program where you want to load the .cfg files from
input_folder = '/content/drive/My Drive/Will_CDI_Project/atomsk_reorient/reoriented_files/'

# Tell the program where you want to save the data to
save_folder = '/content/drive/My Drive/Will_CDI_Project/edge/'

# List all those files
file_list = sorted(os.listdir(input_folder))


for reflection in tqdm(reflections_list):

    for sampling_rate in tqdm(sampling_list):

        for file in tqdm(file_list):
            
            # Obtain the new main file name (main_name),
            #  folder to save everything to (main folder), and save location (fhkl_save_location)
            main_name, main_folder, fhkl_save_location = naming_convention(save_folder,
                                                                           (reflection, sampling_rate, file))

            # See if the save folder exists. If it doesnt then create one
            check_folder_exists(main_folder)

            # Use pynx to simulate patterns
            fhkl, intensity = get_simulation(input_folder=input_folder, 
                                      file=file, hkl=reflection, nstep=sampling_rate)

            # Save patterns
            tif.imsave(fhkl_save_location, np.abs(fhkl))
